In [ ]:
import os, sys 
sys.path.append('/mnt/d/hust/code/thesis')
print(sys.path)
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType
from spark_processing.pipeline.drop_null_columns import drop_null_columns
from spark_processing.pipeline.flatten_dataframe import flatten_dataframe
from delta import *
from spark_processing.pipeline.tgdd.udf import *

['/home/bien/.pyenv/versions/3.9.20/lib/python39.zip', '/tmp/spark-2f4d37dc-9342-4d16-a33d-24f743acf7c6/userFiles-66492fe7-e1d1-49b8-9cc3-952b55eac796/org.antlr_antlr4-runtime-4.9.3.jar', '/tmp/spark-2f4d37dc-9342-4d16-a33d-24f743acf7c6/userFiles-66492fe7-e1d1-49b8-9cc3-952b55eac796/io.delta_delta-storage-3.0.0.jar', '/tmp/spark-2f4d37dc-9342-4d16-a33d-24f743acf7c6/userFiles-66492fe7-e1d1-49b8-9cc3-952b55eac796/io.delta_delta-spark_2.12-3.0.0.jar', '/tmp/spark-2f4d37dc-9342-4d16-a33d-24f743acf7c6/userFiles-66492fe7-e1d1-49b8-9cc3-952b55eac796', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9/lib-dynload', '', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9/site-packages', '/mnt/d/hust/code/thesis', '/mnt/d/hust/code/thesis', '/mnt/d/hust/code/thesis']


In [2]:
import os, sys 
sys.path.append('/mnt/d/hust/code/thesis')
print(sys.path)
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import current_timestamp, date_format
from spark_processing.pipeline.drop_null_columns import drop_null_columns
from spark_processing.pipeline.flatten_dataframe import flatten_dataframe
from spark_processing.pipeline.upsert_normalized_table import upsert_normalized_table

from delta import *
from spark_processing.pipeline.tgdd.udf import *

builder = SparkSession.builder \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()


['/home/bien/.pyenv/versions/3.9.20/lib/python39.zip', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9/lib-dynload', '', '/home/bien/.pyenv/versions/3.9.20/lib/python3.9/site-packages', '/mnt/d/hust/code/thesis']


your 131072x1 screen size is bogus. expect trouble
25/03/01 21:10:05 WARN Utils: Your hostname, DESKTOP-H2N322M resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/01 21:10:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/bienn/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bien/.ivy2/cache
The jars for the packages stored in: /home/bien/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-667eba81-5291-4963-9d4e-5d45c465bfa9;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 346ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

In [ ]:

df = spark.read.format("json").load("/mnt/d/hust/code/thesis/warehouse/daily/tgdd/20250301/*.json")
categories_df = df.select("category").distinct()
category_list = [row['category'] for row in categories_df.collect()]
for category in category_list:
    new_df = df.filter(F.col("category") == category)
    new_df = flatten_dataframe(new_df)
    new_df = drop_null_columns(new_df)
    new_df.show(new_df.count(),truncate=False)
    new_df = new_df.withColumn("updated_at", date_format(current_timestamp(), 'yyyy-MM-dd HH:mm:ss'))
    normalized_table_path = f"/mnt/d/hust/code/thesis/warehouse/categories/tgdd/{category}"
    upsert_normalized_table(spark, new_df, normalized_table_path, 'product_id', 'product_id')

In [3]:
df = spark.read.format("delta").load(f'/mnt/d/hust/code/thesis/warehouse/normalized/tgdd/20250301')
# df = df.select("product_id", "product_name", "category", "url", "brand", "description", "price", "price_origin", "price_present", "crawled_at", "updated_at",
#                 'discount_percent', 'discount', 'is_sale_off', 'source', 'reviews_count')
df.printSchema()
df.show()

25/03/01 21:10:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- bo_nho_luu_tru: struct (nullable = true)
 |    |-- dung_luong_con_lai_khoang: string (nullable = true)
 |    |-- dung_luong_luu_tru: string (nullable = true)
 |    |-- ram: string (nullable = true)
 |    |-- the_nho_ngoai: string (nullable = true)
 |-- bo_nho_ram_o_cung: struct (nullable = true)
 |    |-- chuan_ket_noi_o_cung: string (nullable = true)
 |    |-- ho_tro_ram_toi_da: string (nullable = true)
 |    |-- khe_cam_mo_rong: string (nullable = true)
 |    |-- loai_ram: string (nullable = true)
 |    |-- o_cung: string (nullable = true)
 |    |-- ram: string (nullable = true)
 |    |-- so_khe_ram: string (nullable = true)
 |    |-- tan_nhiet_ram: string (nullable = true)
 |    |-- toc_do_bus_ram: string (nullable = true)
 |-- bo_phat: struct (nullable = true)
 |    |-- dung_luong_pin: string (nullable = true)
 |    |-- khoi_luong: string (nullable = true)
 |    |-- kich_thuoc: string (nullable = true)
 |    |-- loai_pin: string (nullable = true)
 |    |-- nhiet_do_hoat_d

+--------------+-----------------+-------+------+--------+---------+------+---------------+----------+------------+---------------+----+-----------------+---------------+----------------+------------------------------+-----------------------+-------------------+--------------------+------------------+-------+------+------------+----------------+-------+--------------------+-------+--------------------+---------------------+-------------------------+-------------+---------+--------+-----+----+-------+---------+------------+-------------+-----------------+--------------------+--------------------+--------------------+-------------+--------+------------------+--------+--------------------+---------------+------------------+---------------+--------------+------------------+----------+--------+------------------------+-------------------+--------------------+-------------------+----------------+---------+-----------+------+--------------------+--------------------+--------------------+-----